In [233]:
# The following implements the QUBO edges algorithm from the report. 
# Even for small graphs this is a computationally heavy algorithm. 
# The code below serves only as a proof of concept that the algorithm works.

In [234]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [235]:
from pyqubo import Array, LogEncInteger
import neal
from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph
from typing import List

In [236]:
# This function turns a graph into new graph with a tail of specified length.
# All vertices in the original graph are connected to the beginning of the tail.
def make_tail_graph(graph : StandardGraph, tail_length : int) -> StandardGraph:

	if tail_length == 0:
		tail_graph = StandardGraph(0, [])
		tail_graph.vertices += graph.vertices
		tail_graph.edges = graph.edges.copy()
		return tail_graph

	# We set the vertex amount of tail_graph and copy the edges of the original graph into tail_graph.edges.
	tail_graph = StandardGraph(0, [])
	tail_graph.vertices += graph.vertices + tail_length
	tail_graph.edges = graph.edges.copy()
	
	
	# The tail part starts at index graph.vertices.
	# We connect all the original vertices to the beginning of the tail.
	for i in range(0,graph.vertices):
		tail_graph.edges.append((i,graph.vertices))
	
	#We add the connections within the tail.
	for i in range(graph.vertices , graph.vertices + tail_length - 1):
		tail_graph.edges.append((i, i+1))

	return tail_graph


# This functions computes for every vertex in a graph the indices of the edges that are connected to a specific vertex.
# These collections of indices are stored in lists of integers.
# The lists are then stored in an outer list of which the indices correspond to the vertices of the graph.
def make_vertex_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]
		
		out[v_1].append(i)
		out[v_2].append(i)
	
	return out

# We endow an orientation on all the edged in a graph.
# Edge (i,j) goes from i to j.
# With respect to this orientation, we compute for each vertex in a graph which edges point towards that vertex
def make_in_flow_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(_, v_2) = graph.edges[i]

		out[v_2].append(i)
	
	return out

#We also compute, for each vertex, which edges point away from that vertex
def make_out_flow_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, _) = graph.edges[i]

		out[v_1].append(i)
	
	return out



In [237]:

# We set a graph of which we want to compute the longest path.
# We were not able to solve graphs with more that 3 vertices within 15 minutes.
graph = StandardGraph(5, [
	(0,1),
    (1,2),
    (0,2),
    (0,3),
    (0,4)
])

# Use this line to generate a random graph
# graph = gen_planted_path(10, 0.05)

# N is the amount of vertices in the graph
N = graph.vertices

# K is the maximal length of a path that the algorithm can find
K = N - 1

# P is the penalty value that we use for the QUBO algorithm
P = -N

# We add a tail to the original graph as is described in the report.
tail_graph = make_tail_graph(graph, K)

# For each vertex we compute which edges are connected to it.
tail_vertex_edges = make_vertex_edges(tail_graph)

# We view all edges as oriented edges and store for each vertex
# the ingoing and outgoing edges.
out_flow_edges = make_out_flow_edges(tail_graph)
in_flow_edges = make_in_flow_edges(tail_graph)


# We print information about the input graph and the constructed graph with tail.

print("Original graph:")
print("vertices : ", graph.vertices)
print("edges : ", graph.edges)
print()

print("Graph with tail:")
print("vertices : ", tail_graph.vertices)
print("edges : ", tail_graph.edges)
print()

print("Vertex : Edges connected to vertex in graph with tail")
for i in range(len(tail_vertex_edges)):
    print(f"{i} : {tail_vertex_edges[i]}")


Original graph:
vertices :  5
edges :  [(0, 1), (1, 2), (0, 2), (0, 3), (0, 4)]

Graph with tail:
vertices :  9
edges :  [(0, 1), (1, 2), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 6), (6, 7), (7, 8)]

Vertex : Edges connected to vertex in graph with tail
0 : [0, 2, 3, 4, 5]
1 : [0, 1, 6]
2 : [1, 2, 7]
3 : [3, 8]
4 : [4, 9]
5 : [5, 6, 7, 8, 9, 10]
6 : [10, 11]
7 : [11, 12]
8 : [12]


In [238]:

# Let E be the amount of edges in the input graph.
# We formulate the problem as a maximization problem.

# We create for each vertex a binary variable that indicates if that vertex is on the path.

vertex_vars = Array.create('vertex', tail_graph.vertices, vartype='BINARY') #N+K binary variables

# For each edge in the graph with tail, we create a binary value that indicates if that edge is used in the path.
# We also consider edges from an initial and terminal "vertex". The values of these variables indicate where a path begins and ends.

edge_vars = Array.create('edge', len(tail_graph.edges), vartype='BINARY') # E + N + K - 1 binary variables
initial_edge_vars = Array.create('init_edge', graph.vertices, vartype='BINARY') # N binary variables
terminal_edge_vars = Array.create('term_edge', tail_graph.vertices, vartype='BINARY') # N+K binary variables

# For each edge (with initial and terminal edges included) we store a inetger flow value in the possitive dirrection of the edges.
# A flow value can be at most K + 2 and must be at least 0.

flow_vars = [LogEncInteger(f"flow_vars[{i}]", (0,K + 2)) for i in range(len(tail_graph.edges))] # (E + N + K - 1) * (floor(log_2(K+2)) + 1) binary variables
initial_flow_vars = [LogEncInteger(f"initial_flow_vars[{i}]", (0,K + 2)) for i in range(graph.vertices)] #assumed to be ingoing, (N) * (floor(log_2(K+2)) + 1) binary variables
terminal_flow_vars = [LogEncInteger(f"terminal_flow_vars[{i}]", (0,K + 2)) for i in range(tail_graph.vertices)] #assumed to be outgoing, (N + K) * (floor(log_2(K+2)) + 1) binary variables

# We also store flow values for the negative directions of edges.

contra_flow_vars = [LogEncInteger(f"contra_flow_vars[{i}]", (0,K + 2)) for i in range(len(tail_graph.edges))] # (E + N + K - 1) * (floor(log_2(K+2)) + 1) binary variables
initial_contra_flow_vars = [LogEncInteger(f"initial_contra_flow_vars[{i}]", (0,K + 2)) for i in range(graph.vertices)] #assumed to be outgoing, (N) * (floor(log_2(K+2)) + 1) binary variables
terminal_contra_flow_vars = [LogEncInteger(f"terminal_contra_flow_vars[{i}]", (0,K + 2)) for i in range(tail_graph.vertices)] #assumed to be ingoing, (N + K) * (floor(log_2(K+2)) + 1) binary variables

# The total variable count is:
# (N+K) + (E+N+K-1) + (N) + (N+K) + (E + N + K - 1) * (floor(log_2(K+2)) + 1) + (N) * (floor(log_2(K+2)) + 1) + (N + K) * (floor(log_2(K+2)) + 1) + (E + N + K - 1) * (floor(log_2(K+2)) + 1) + (N) * (floor(log_2(K+2)) + 1) + (N + K) * (floor(log_2(K+2)) + 1)
# = (E + 4N + 3K - 2) + (2E + 6N + 4K - 2) * (floor(log_2(K+2)) + 1)
# = (3E + 10N + 7K - 4) + (2E + 6N + 4K - 2) * floor(log_2(K+2))
# Usually K = N - 1, so then this becomes
# = (3E + 17N - 11) + (2E + 10N + 6) * floor(log_2(N + 1))
# Where E is at most N^2.

# This variable will store the matrix for the qubo formulation in the form of a quadratic expression.
qubo_matrix_expression = 0

#This part of the expression rewards longer paths which 1 point per used edge.
candy_exp = sum(edge_vars[i] for i in range(len(graph.edges)))
qubo_matrix_expression += candy_exp

# This part of the expression results in a penalty if multiple initial edges are used.
initial_node_exp = P * (1 - sum(initial_edge_vars[i] for i in range(graph.vertices)))**2
qubo_matrix_expression += initial_node_exp

# This part of the expression results in a penalty if multiple terminal edges are used.
terminal_node_exp = P * (1 - sum(terminal_edge_vars[i] for i in range(tail_graph.vertices)))**2
qubo_matrix_expression += terminal_node_exp

# This part of the expression results in a penalty if multiple vertices are connected to the beginning of the tail.
fake_terminal_node_exp_1 = P * (vertex_vars[graph.vertices] - sum(edge_vars[i] for i in range( len(graph.edges), len(graph.edges) + graph.vertices)))**2
qubo_matrix_expression += fake_terminal_node_exp_1

# This part of the expression results in a penalty if second vertex in the tail is used without using the beginning of the tail.
fake_terminal_node_exp_2 = P * (1 - vertex_vars[graph.vertices]) * vertex_vars[graph.vertices + 1]
qubo_matrix_expression += fake_terminal_node_exp_2

# This part of the expression results in a penalty if there are not exactly 2 edges used that are connected to the same vertex while that vertex lies on the path.
# If a vertex does not lie on the path it results in a penalty if any more than 0 connected edges are used.
# We also consider the terminal and intial edges in this case.
real_node_deg_2_exp = 0
for i in range(tail_graph.vertices):
	temp1 = sum(edge_vars[tail_vertex_edges[i][j]] for j in range(len(tail_vertex_edges[i])))
	
	temp1 += terminal_edge_vars[i]

	# Only vertices from the input graph are connected to initial edges
	if i < graph.vertices:
		temp1 += initial_edge_vars[i]

	# For vertex i we make sure that exactly 0 or 2 connected edges are used (depending on if the vertex lies on the path)
	real_node_deg_2_exp += P * (2 * vertex_vars[i] - temp1)**2

qubo_matrix_expression += real_node_deg_2_exp

#This part of the expression results in a penalty if the flow and contra flow values of any edge that is used do not add up to K + 3.
pairing_flow_exp = P * sum([((K + 3) * edge_vars[i] - flow_vars[i] - contra_flow_vars[i])**2 for i in range(len(edge_vars))])

pairing_flow_exp += P * sum([((K + 3) * initial_edge_vars[i] - initial_flow_vars[i] - initial_contra_flow_vars[i])**2 for i in range(len(initial_edge_vars))])

pairing_flow_exp += P * sum([((K + 3) * terminal_edge_vars[i] - terminal_flow_vars[i] - terminal_contra_flow_vars[i])**2 for i in range(len(terminal_edge_vars))])
	
qubo_matrix_expression += pairing_flow_exp

# This part of the expression makes sure that value of the initial flow and the value of the contra terminal flow are both equal to 1.
set_initial_flow_exp = P * (1 - sum([a for a in initial_flow_vars]))**2
set_terminal_flow_exp = P * (1 - sum(b for b in terminal_contra_flow_vars))**2

qubo_matrix_expression += set_initial_flow_exp + set_terminal_flow_exp

# This part of the expression results in a penalty if the total ingoing flow in any vertex that lies on the path is unequal to K + 2
# For vertices that do not lie on the path the expression results in a penalty if the total ingoing flow is unequal to 0.
set_out_flow_exp = 0
for i in range(tail_graph.vertices):
	#For each vertex we set the value that the ingoing flow must be
	temp_exp = (K+2) * vertex_vars[i]

	# For each connected vertex we subtract the corresponging ingoing flow value
	if i < graph.vertices:
		temp_exp += -initial_flow_vars[i]
	temp_exp += -terminal_contra_flow_vars[i]
	temp_exp += -sum([contra_flow_vars[j] for j in out_flow_edges[i]])
	temp_exp += -sum([flow_vars[j] for j in in_flow_edges[i]])

	temp_exp = temp_exp**2

	set_out_flow_exp += P * temp_exp	

qubo_matrix_expression += set_out_flow_exp


In [239]:
"""
The variables in the block contain solutions for the longest paths in example graphs.
"""

"""
Solution for graph: 
vertices: 0, 1, 2
edges: (0,1), (1,2)
"""
line_sample = {'edge[2]': 0, 'contra_flow_vars[0][0]': 0, 'edge[3]': 0, 'contra_flow_vars[0][1]': 1, 'edge[0]': 1, 'contra_flow_vars[0][2]': 1, 'contra_flow_vars[1][0]': 0, 'contra_flow_vars[2][1]': 0, 'initial_flow_vars[1][2]': 0, 'flow_vars[0][2]': 1, 'contra_flow_vars[1][1]': 1, 'contra_flow_vars[2][0]': 0, 'initial_flow_vars[1][1]': 0, 'flow_vars[0][1]': 0, 'contra_flow_vars[1][2]': 0, 'initial_flow_vars[1][0]': 0, 'flow_vars[0][0]': 1, 'contra_flow_vars[2][2]': 0, 'contra_flow_vars[3][0]': 0, 'contra_flow_vars[3][1]': 0, 'contra_flow_vars[3][2]': 0, 'contra_flow_vars[4][0]': 0, 'terminal_flow_vars[0][1]': 0, 'term_edge[0]': 0, 'contra_flow_vars[4][1]': 0, 'terminal_flow_vars[0][0]': 0, 'edge[4]': 0, 'contra_flow_vars[4][2]': 0, 'initial_flow_vars[2][2]': 0, 'flow_vars[3][2]': 0, 'init_edge[2]': 0, 'contra_flow_vars[5][0]': 0, 'contra_flow_vars[5][1]': 0, 'initial_flow_vars[2][0]': 0, 'flow_vars[3][0]': 0, 'terminal_flow_vars[3][1]': 0, 'init_edge[0]': 1, 'contra_flow_vars[5][2]': 0, 'edge[1]': 1, 'edge[5]': 0, 'initial_flow_vars[0][0]': 1, 'flow_vars[1][0]': 1, 'flow_vars[2][1]': 0, 'initial_flow_vars[0][1]': 0, 'flow_vars[1][1]': 1, 'flow_vars[2][0]': 0, 'initial_flow_vars[0][2]': 0, 'flow_vars[1][2]': 0, 'flow_vars[2][2]': 0, 'initial_flow_vars[2][1]': 0, 'flow_vars[3][1]': 0, 'terminal_flow_vars[3][0]': 0, 'init_edge[1]': 0, 'flow_vars[4][0]': 0, 'flow_vars[4][1]': 0, 'flow_vars[4][2]': 0, 'flow_vars[5][0]': 0, 'flow_vars[5][1]': 0, 'flow_vars[5][2]': 0, 'initial_contra_flow_vars[0][0]': 1, 'initial_contra_flow_vars[0][1]': 1, 'initial_contra_flow_vars[0][2]': 1, 'initial_contra_flow_vars[2][1]': 0, 'initial_contra_flow_vars[1][0]': 0, 'initial_contra_flow_vars[2][0]': 0, 'initial_contra_flow_vars[1][1]': 0, 'initial_contra_flow_vars[1][2]': 0, 'initial_contra_flow_vars[2][2]': 0, 'term_edge[1]': 0, 'term_edge[2]': 1, 'term_edge[3]': 0, 'term_edge[4]': 0, 'terminal_contra_flow_vars[0][0]': 0, 'terminal_contra_flow_vars[0][1]': 0, 'terminal_contra_flow_vars[0][2]': 0, 'terminal_contra_flow_vars[1][0]': 0, 'terminal_contra_flow_vars[1][1]': 0, 'terminal_contra_flow_vars[1][2]': 0, 'terminal_contra_flow_vars[2][0]': 1, 'terminal_contra_flow_vars[2][1]': 0, 'terminal_contra_flow_vars[2][2]': 0, 'terminal_contra_flow_vars[3][0]': 0, 'terminal_contra_flow_vars[3][1]': 0, 'terminal_contra_flow_vars[3][2]': 0, 'terminal_contra_flow_vars[4][0]': 0, 'terminal_contra_flow_vars[4][1]': 0, 'terminal_contra_flow_vars[4][2]': 0, 'terminal_flow_vars[0][2]': 0, 'terminal_flow_vars[1][0]': 0, 'terminal_flow_vars[1][1]': 0, 'terminal_flow_vars[1][2]': 0, 'terminal_flow_vars[2][0]': 1, 'terminal_flow_vars[2][1]': 1, 'terminal_flow_vars[2][2]': 1, 'terminal_flow_vars[3][2]': 0, 'terminal_flow_vars[4][0]': 0, 'terminal_flow_vars[4][1]': 0, 'terminal_flow_vars[4][2]': 0, 'vertex[0]': 1, 'vertex[1]': 1, 'vertex[2]': 1, 'vertex[3]': 0, 'vertex[4]': 0}


"""
Solution for graph: 
vertices: 0, 1, 2
edges: (0,1), (1,2), (0,2)
"""
triangle_sample = {'edge[2]': 1, 'contra_flow_vars[0][0]': 0, 'edge[3]': 0, 'contra_flow_vars[0][1]': 0, 'edge[0]': 0, 'contra_flow_vars[0][2]': 0, 'contra_flow_vars[1][0]': 1, 'contra_flow_vars[2][1]': 1, 'flow_vars[0][2]': 0, 'initial_flow_vars[1][2]': 0, 'contra_flow_vars[1][1]': 1, 'contra_flow_vars[2][0]': 0, 'flow_vars[0][1]': 0, 'initial_flow_vars[1][1]': 0, 'contra_flow_vars[1][2]': 0, 'flow_vars[0][0]': 0, 'initial_flow_vars[1][0]': 0, 'contra_flow_vars[2][2]': 1, 'contra_flow_vars[3][0]': 0, 'contra_flow_vars[3][1]': 0, 'contra_flow_vars[3][2]': 0, 'edge[6]': 0, 'contra_flow_vars[4][0]': 0, 'terminal_flow_vars[0][1]': 0, 'contra_flow_vars[4][1]': 0, 'terminal_flow_vars[0][0]': 0, 'edge[4]': 0, 'contra_flow_vars[4][2]': 0, 'init_edge[2]': 0, 'flow_vars[3][2]': 0, 'initial_flow_vars[2][2]': 0, 'contra_flow_vars[6][1]': 0, 'contra_flow_vars[5][0]': 0, 'contra_flow_vars[6][0]': 0, 'contra_flow_vars[5][1]': 0, 'init_edge[0]': 1, 'flow_vars[3][0]': 0, 'initial_flow_vars[2][0]': 0, 'terminal_flow_vars[3][1]': 0, 'contra_flow_vars[5][2]': 0, 'init_edge[1]': 0, 'flow_vars[3][1]': 0, 'initial_flow_vars[2][1]': 0, 'terminal_flow_vars[3][0]': 0, 'contra_flow_vars[6][2]': 0, 'edge[1]': 1, 'edge[5]': 0, 'initial_flow_vars[0][0]': 1, 'flow_vars[1][0]': 1, 'flow_vars[2][1]': 0, 'initial_flow_vars[0][1]': 0, 'flow_vars[1][1]': 0, 'flow_vars[2][0]': 1, 'initial_flow_vars[0][2]': 0, 'flow_vars[1][2]': 1, 'flow_vars[2][2]': 1, 'flow_vars[4][0]': 0, 'flow_vars[4][1]': 0, 'flow_vars[4][2]': 0, 'flow_vars[6][1]': 0, 'flow_vars[5][0]': 0, 'flow_vars[6][0]': 0, 'flow_vars[5][1]': 0, 'flow_vars[5][2]': 0, 'flow_vars[6][2]': 0, 'initial_contra_flow_vars[0][0]': 1, 'initial_contra_flow_vars[0][1]': 1, 'initial_contra_flow_vars[0][2]': 1, 'initial_contra_flow_vars[2][1]': 0, 'initial_contra_flow_vars[1][0]': 0, 'initial_contra_flow_vars[2][0]': 0, 'initial_contra_flow_vars[1][1]': 0, 'initial_contra_flow_vars[1][2]': 0, 'initial_contra_flow_vars[2][2]': 0, 'term_edge[0]': 0, 'term_edge[1]': 1, 'term_edge[2]': 0, 'term_edge[3]': 0, 'term_edge[4]': 0, 'terminal_contra_flow_vars[0][0]': 0, 'terminal_contra_flow_vars[0][1]': 0, 'terminal_contra_flow_vars[0][2]': 0, 'terminal_contra_flow_vars[1][0]': 0, 'terminal_contra_flow_vars[1][1]': 0, 'terminal_contra_flow_vars[1][2]': 1, 'terminal_contra_flow_vars[2][0]': 0, 'terminal_contra_flow_vars[2][1]': 0, 'terminal_contra_flow_vars[2][2]': 0, 'terminal_contra_flow_vars[3][0]': 0, 'terminal_contra_flow_vars[3][1]': 0, 'terminal_contra_flow_vars[3][2]': 0, 'terminal_contra_flow_vars[4][0]': 0, 'terminal_contra_flow_vars[4][1]': 0, 'terminal_contra_flow_vars[4][2]': 0, 'terminal_flow_vars[0][2]': 0, 'terminal_flow_vars[1][0]': 1, 'terminal_flow_vars[1][1]': 1, 'terminal_flow_vars[1][2]': 1, 'terminal_flow_vars[2][0]': 0, 'terminal_flow_vars[2][1]': 0, 'terminal_flow_vars[2][2]': 0, 'terminal_flow_vars[3][2]': 0, 'terminal_flow_vars[4][0]': 0, 'terminal_flow_vars[4][1]': 0, 'terminal_flow_vars[4][2]': 0, 'vertex[0]': 1, 'vertex[1]': 1, 'vertex[2]': 1, 'vertex[3]': 0, 'vertex[4]': 0}




In [240]:
# We take the negative version of the expression to turn the maximization problem into a minimization problem.
qubo_matrix_expression = - qubo_matrix_expression


# We turn the qubo expression into a model and solve it with simmulated annealing with the pyqubo (actually dimod) library
model = qubo_matrix_expression.compile()
bqm = model.to_bqm()
sa = neal.SimulatedAnnealingSampler()

#Use the upper line to input a solution
#sampleset = sa.sample(bqm, num_reads=1, num_sweeps=0, initial_states= triangle_sample)
sampleset = sa.sample(bqm, num_reads=10000, num_sweeps=100, beta_range=(0.1, 2))

decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)

In [241]:
# We print the length of the longuest path that we found and the corresponding sample
print("Best path length = ", -best_sample.energy)
print("Corresponding solution =", best_sample.sample)
print()

# We print what the graph with tail looks like.
print("Graph with tail:")
print("vertices : ", tail_graph.vertices)
print("edges : ", tail_graph.edges)
print()

# We print the information in the sample in a way that makes debugging easier
print("Vertices:")
for i in range(tail_graph.vertices):
	print(f"vertex {i} : ", best_sample.sample[f"vertex[{i}]"])

print()
print("Edges:")

for i in range(graph.vertices):
	print(f"Initial edge {i} : ", best_sample.sample[f"init_edge[{i}]"])

for i in range(len(tail_graph.edges)):
	print(f"Edge {i} : ", best_sample.sample[f"edge[{i}]"])

for i in range(tail_graph.vertices):
	print(f"Terminal edge {i} : ", best_sample.sample[f"term_edge[{i}]"])

print()

print("Flow variables:")
for i in range(graph.vertices):	
	print(f"Initial flow var {i}: ", best_sample.subh[f"initial_flow_vars[{i}]"])

for i in range(len(tail_graph.edges)):	
	print(f"Flow var {i}: ", best_sample.subh[f"flow_vars[{i}]"])

for i in range(tail_graph.vertices):
	print(f"Terminal flow var {i}: ", best_sample.subh[f"terminal_flow_vars[{i}]"])

print()

print("Contra flow variables:")
for i in range(graph.vertices):	
	print(f"Initial contra flow var {i}: ", best_sample.subh[f"initial_contra_flow_vars[{i}]"])

for i in range(len(tail_graph.edges)):	
	print(f"Contra flow var {i}: ", best_sample.subh[f"contra_flow_vars[{i}]"])

for i in range(tail_graph.vertices):	
	print(f"Terminal contra flow var {i}: ", best_sample.subh[f"terminal_contra_flow_vars[{i}]"])

Best path length =  -38.0
Corresponding solution = {'contra_flow_vars[7][1]': 0, 'flow_vars[4][0]': 0, 'contra_flow_vars[11][0]': 0, 'edge[2]': 0, 'contra_flow_vars[0][0]': 1, 'contra_flow_vars[7][0]': 1, 'flow_vars[4][1]': 0, 'contra_flow_vars[11][1]': 1, 'edge[3]': 0, 'contra_flow_vars[0][1]': 0, 'edge[0]': 1, 'contra_flow_vars[0][2]': 1, 'contra_flow_vars[10][0]': 0, 'contra_flow_vars[10][1]': 0, 'contra_flow_vars[10][2]': 1, 'init_edge[4]': 1, 'contra_flow_vars[11][2]': 0, 'contra_flow_vars[12][0]': 1, 'initial_contra_flow_vars[2][2]': 0, 'terminal_flow_vars[8][0]': 0, 'contra_flow_vars[12][1]': 0, 'terminal_flow_vars[8][1]': 0, 'contra_flow_vars[12][2]': 0, 'initial_contra_flow_vars[2][0]': 0, 'terminal_flow_vars[8][2]': 0, 'contra_flow_vars[2][1]': 0, 'contra_flow_vars[1][0]': 0, 'flow_vars[0][2]': 0, 'initial_flow_vars[1][2]': 0, 'contra_flow_vars[2][0]': 1, 'terminal_contra_flow_vars[5][2]': 0, 'contra_flow_vars[1][1]': 0, 'flow_vars[0][1]': 1, 'initial_flow_vars[1][1]': 0, 'co